In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
import io
%cd "C:\Users\hp\Downloads"

C:\Users\hp\Downloads


In [3]:
rfm_anal = pd.read_csv("rfm_data.csv")

In [4]:
rfm_anal

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris
...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London
996,6669,2023-06-10,941.50,Product C,987025,New York
997,8836,2023-06-10,545.36,Product C,512842,London
998,1440,2023-06-10,729.94,Product B,559753,Paris


In [5]:
rfm_anal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


In [6]:
### RFM is used for Customer Segmentation
### MBA (Market basket analysis) is used for Product Segmentation

In [7]:
rfm_anal["PurchaseDate"] = pd.to_datetime(rfm_anal["PurchaseDate"])

In [8]:
rfm_anal

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris
...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London
996,6669,2023-06-10,941.50,Product C,987025,New York
997,8836,2023-06-10,545.36,Product C,512842,London
998,1440,2023-06-10,729.94,Product B,559753,Paris


In [9]:
rfm_anal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   int64         
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   int64         
 5   Location            1000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 47.0+ KB


In [10]:
from datetime import datetime

In [11]:
datetime.now().date()

datetime.date(2024, 7, 14)

In [12]:
### Calculate recency

rfm_anal['Recency'] = (datetime.now().date() - rfm_anal.PurchaseDate.dt.date)/np.timedelta64(1, 'D')   ### Time delta is Time Difference

In [13]:
## Calculate Frequency

freq_df = rfm_anal.groupby('CustomerID')['OrderID'].count().reset_index()

freq_df.rename(columns={'OrderID' : 'Frequency'}, inplace=True)

In [14]:
freq_df.head()

,CustomerID,OrderID
0,1011,2
1,1025,1
2,1029,1
3,1046,1
4,1049,1


In [15]:
freq_df.rename(columns={'OrderID':'Frequency'}, inplace=True)

In [16]:
## Add freq to dataframe
rfm_anal = rfm_anal.merge(freq_df, on = 'CustomerID', how = "left")

In [17]:
rfm_anal.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,460.0,1
1,2188,2023-04-11,463.70,Product A,176819,London,460.0,1
2,4608,2023-04-11,80.28,Product A,340062,New York,460.0,1
3,2559,2023-04-11,221.29,Product A,239145,London,460.0,1
4,9482,2023-04-11,739.56,Product A,194545,Paris,460.0,1


In [18]:
## Calculate Monatory value

monetary = rfm_anal.groupby('CustomerID')['TransactionAmount'].sum().reset_index()

In [19]:
monetary.rename(columns={'TransactionAmount':'MonetaryValue'}, inplace=True)

In [20]:
rfm_anal=rfm_anal.merge(monetary, on='CustomerID',how="left")

In [21]:
#Define scales or score for RFM
recency_score = [5,4,3,2,1]
frequency_score = [1,2,3,4,5]
monetary_score = [1,2,3,4,5]

In [22]:
# Calculate the Recency score for RFM
rfm_anal['Recency_score'] = pd.cut(rfm_anal.Recency, bins=5, labels=recency_score)

In [25]:
rfm_anal['Frequency_score'] = pd.cut(rfm_anal.Frequency, bins=5, labels=frequency_score)

In [26]:
rfm_anal['Monetary_score'] = pd.cut(rfm_anal.MonetaryValue, bins=5, labels=monetary_score)

In [27]:
## Change the datatype to int for recency, frequency, monetary
rfm_anal.Recency_score = rfm_anal.Recency_score.astype(int)
rfm_anal.Monetary_score = rfm_anal.Monetary_score.astype(int)
rfm_anal.Frequency_score = rfm_anal.Frequency_score.astype(int)

In [28]:
rfm_anal['RFM_Score'] = rfm_anal.Recency_score+rfm_anal.Frequency_score+rfm_anal.Monetary_score

In [29]:
# Create a new variable customer segment
segment_labels = ["Low Value", "Medium Value", "High Value"]
rfm_anal['Customer_Segment'] = pd.qcut(rfm_anal.RFM_Score, q=3, labels=segment_labels)

In [30]:
rfm_anal.Customer_Segment.value_counts()

Customer_Segment
Low Value       435
Medium Value    386
High Value      179
Name: count, dtype: int64

In [31]:
rfm_anal.RFM_Score.value_counts()

RFM_Score
7     196
6     190
5     180
4     173
8     117
3      82
9      28
11     15
10     12
12      4
13      2
15      1
Name: count, dtype: int64

In [34]:
rfm_segments = []
for row in rfm_anal.RFM_Score:
    if row>=9:
        rfm_segments.append("Champions")
    elif row>=6:
        rfm_segments.append("Loyalist")
    elif row>=5:
        rfm_segments.append("At Risk")
    elif row>=4:
        rfm_segments.append("Cant Loose")
    else:
        rfm_segments.append("Lost")
rfm_anal['RFM_Segment'] = rfm_segments

In [35]:
rfm_anal.RFM_Segment.value_counts()

RFM_Segment
Loyalist      503
At Risk       180
Cant Loose    173
Lost           82
Champions      62
Name: count, dtype: int64